<a href="https://colab.research.google.com/github/componavt/differential_equations/blob/main/src/hill_equation/99_DOP853_round3_float32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬📊 Solution Archive for Gene Regulatory ODE System (Compressed) 🚀

**Overview**  
This notebook computes numerical solutions of a gene-regulatory ODE system for many parameter combinations, compresses trajectories using a delta-based filter, and stores all results in a single compact file. Designed for fast and interactive visualization (e.g., in Streamlit) ⚡️.

---

**Model Equations**

\[
\begin{cases}
\frac{dx}{dt} = \frac{K\,x^{1/\alpha}}{b^{1/\alpha} + x^{1/\alpha}} \;-\; \gamma_1\,x,\\[6pt]
\frac{dy}{dt} = \frac{K\,y^{1/\alpha}}{b^{1/\alpha} + y^{1/\alpha}} \;-\; \gamma_2\,y.
\end{cases}
\]

**Fixed parameters**  
- b = 1.0, K = 1.0 ⚙️  
- Time: t ∈ [0, 1.0], N = 500 steps ⏳  
- Initial conditions: x₀ = y₀ = b × (1 + shift), where shift ∈ {−1e−3, −1e−6, 0, 1e−6, 1e−3} 🎯

**Parameter sweeps**  
- α ∈ {1e−9, 1e−10, ..., 1e−14} 🔄  
- γ₁, γ₂ ∈ {0.0, 0.1, ..., 1.0} 🔄  
- Solvers: DOP853, BDF, Radau, RK23 🧮

---

**Compression algorithm** 🗜️  
After solving the ODE:  
- Start from the first point 🚩  
- Keep the next point only if the Euclidean distance in (x, y) exceeds delta = 1e−6 ➡️  
- Always keep the first and last points 🎯  
- This drastically reduces data size while preserving visual fidelity 👁️‍🗨️

---

**Each row in the final DataFrame contains**  
- Parameters: x₀, y₀, γ₁, γ₂, α 📊  
- t: filtered time array (shared across solvers) ⏲️  
- solutions: a dictionary `{method_name: (x_array, y_array)}` 🗂️

All results are saved to:  
📁 `ode_solutions/solutions.pkl` 💾

---

**How to use this in Streamlit**  
- Load the DataFrame with `pd.read_pickle(...)` 📥  
- Let users select α, γ₁, γ₂, and view solver comparisons 🎛️  
- Plot filtered trajectories for better performance ⚡️

---

*This notebook provides a compact and scalable pipeline for collecting and visualizing large numbers of ODE simulations 🚀*

In [21]:
# Modified code for saving ODE solutions with trajectory compression using delta filtering
# Cell 1: Parameters and library imports
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import os

# Model parameters
b = 1.0
t_end = 1.0
K = 1.0
#alpha_list = [1e-9 * (10 ** -i) for i in range(6)]
alpha_list = [1e-3 * (10 ** -i) for i in range(4)]

# Time discretization
N = 500
t_eval = np.linspace(0, t_end, N)

# Solver methods and gamma parameters
#all_methods = ["DOP853", "BDF", "Radau", "RK23"]
method = "DOP853"
gamma_1_list = np.arange(0, 1.01, 0.1)
gamma_2_list = np.arange(0, 1.01, 0.1)

# Initial conditions: points on a circle around (b, b)
initial_radius = 0.01
num_points = 25
angles = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
initial_conditions = [(b + initial_radius * np.cos(a), b + initial_radius * np.sin(a)) for a in angles]

# Output folder
output_folder = "ode_solutions"
os.makedirs(output_folder, exist_ok=True)

# Delta threshold for filtering
# delta = 1e-6  # ~0.99% reduction
# delta = 1e-3  # ~19.02% reduction
# delta = 5e-3  # ~77.16% reduction
# delta = 1e-2  # ~87.60% reduction
delta = 5e-2   # ~97.15% reduction

def compress_trajectory(t, x, y, delta):
    indices = [0]
    i = 0
    while i < len(t) - 1:
        j = i + 1
        while j < len(t):
            dist = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2)
            if dist > delta:
                break
            j += 1
        if j >= len(t):
            break
        indices.append(j)
        i = j
    if indices[-1] != len(t) - 1:
        indices.append(len(t) - 1)
    return indices

In [22]:
# Cell 2: Define RHS function
def get_rhs(alpha, gamma1, gamma2):
    def rhs(t, xy):
        x = np.clip(xy[0], 1e-20, 1e20)
        y = np.clip(xy[1], 1e-20, 1e20)
        inv_alpha = 1.0 / alpha
        x_alpha = np.exp(np.clip(np.log(x) * inv_alpha, -700, 700))
        y_alpha = np.exp(np.clip(np.log(y) * inv_alpha, -700, 700))
        b_alpha = np.exp(np.clip(np.log(b) * inv_alpha, -700, 700))
        dxdt = K * x_alpha / (b_alpha + x_alpha) - gamma1 * x
        dydt = K * y_alpha / (b_alpha + y_alpha) - gamma2 * y
        return [dxdt, dydt]
    return rhs

In [23]:
# Cell 3: Solve and compress
records = []
solution_count = 0
skipped_total = 0
kept_total = 0

for x0, y0 in initial_conditions:
    for gamma1 in gamma_1_list:
        for gamma2 in gamma_2_list:
            for alpha in alpha_list:
                rhs = get_rhs(alpha, gamma1, gamma2)
                try:
                    sol = solve_ivp(rhs, [0, t_end], [x0, y0], t_eval=t_eval, method=method)
                    if sol.success:
                        x = sol.y[0]
                        y = sol.y[1]
                        t = sol.t
                        indices = compress_trajectory(t, x, y, delta)

                        x_filt = np.round(x[indices], 3).astype(np.float32)
                        y_filt = np.round(y[indices], 3).astype(np.float32)
                        t_filt = np.round(t[indices], 3).astype(np.float32)

                        kept_total += len(indices)
                        skipped_total += len(t) - len(indices)

                        records.append({
                            'x0': x0,
                            'y0': y0,
                            'gamma1': gamma1,
                            'gamma2': gamma2,
                            'alpha': alpha,
                            't': t_filt,
                            'x': x_filt,
                            'y': y_filt
                        })

                        solution_count += 1
                        if solution_count % 1000 == 0:
                            compact = ','.join(map(str, indices[:5])) + ('...' if len(indices) > 5 else '')
                            xvals = ', '.join(f"{v:.3g}" for v in x_filt[:5]) + ('...' if len(x_filt) > 5 else '')#
                            yvals = ', '.join(f"{v:.3g}" for v in y_filt[:5]) + ('...' if len(y_filt) > 5 else '')#
                            print(f"#{solution_count}: x0={x0:.1e}, alpha={alpha:.1e}, γ1={gamma1:.2f}, γ2={gamma2:.2f}, {method}, points={len(indices)}, kept={compact}")
                            print(f"  x: {xvals}\n  y: {yvals}\n")#
                except Exception as e:
                    print(f"Error: method={method}, alpha={alpha}, gamma1={gamma1}, gamma2={gamma2}, x0={x0}: {e}")

#1000: x0=1.0e+00, alpha=1.0e-06, γ1=0.00, γ2=0.70, DOP853, points=21, kept=0,24,49,74,99...
  x: 1.01, 1.06, 1.11, 1.16, 1.21...
  y: 1, 1.02, 1.03, 1.05, 1.06...

#2000: x0=1.0e+00, alpha=1.0e-06, γ1=0.10, γ2=0.40, DOP853, points=21, kept=0,24,48,72,96...
  x: 1, 1.05, 1.09, 1.13, 1.18...
  y: 1.01, 1.04, 1.07, 1.09, 1.12...

#3000: x0=1.0e+00, alpha=1.0e-06, γ1=0.20, γ2=0.10, DOP853, points=23, kept=0,21,42,64,86...
  x: 1, 1.03, 1.07, 1.1, 1.14...
  y: 1.01, 1.05, 1.09, 1.12, 1.16...

#4000: x0=1.0e+00, alpha=1.0e-06, γ1=0.20, γ2=0.90, DOP853, points=5, kept=0,119,246,383,499
  x: 0.996, 0.949, 0.902, 0.854, 0.815
  y: 1.01, 1.03, 1.05, 1.06, 1.07

#5000: x0=9.9e-01, alpha=1.0e-06, γ1=0.30, γ2=0.60, DOP853, points=9, kept=0,52,107,165,226...
  x: 0.992, 0.961, 0.93, 0.898, 0.866...
  y: 1.01, 1.05, 1.09, 1.12, 1.16...

#6000: x0=9.9e-01, alpha=1.0e-06, γ1=0.40, γ2=0.30, DOP853, points=15, kept=0,32,65,98,132...
  x: 0.99, 0.965, 0.94, 0.915, 0.891...
  y: 1, 1.05, 1.09, 1.13, 1.18.

In [24]:
# Cell 4: Save
df = pd.DataFrame(records)
df.to_pickle(os.path.join(output_folder, "solutions.pkl"))
print(f"Saved DataFrame with {len(df)} rows to 'solutions.pkl'")
print(f"Total filtered points kept: {kept_total}, skipped: {skipped_total}, percent removed: {100.0 * skipped_total / (kept_total + skipped_total):.2f}%")

Saved DataFrame with 12100 rows to 'solutions.pkl'
Total filtered points kept: 171098, skipped: 5878902, percent removed: 97.17%
